In [25]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import scipy.signal
from scipy import fftpack, hamming
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn import svm, metrics, preprocessing
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn import grid_search
from sklearn.metrics import make_scorer

データの抽出

In [26]:
def  input(x):
    a1, a2, a3, a4, a5, a6, a7, a8, a9,a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31, a32, a33, a34, a35, a36, a37, a38, a39, a40, a41, a42, a43, a44, a45 = [x.iloc[250*3*i:256+250*3*i,0:8] for i in range(45)]
    return (a1, a2, a3, a4, a5, a6, a7, a8, a9,a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31, a32, a33, a34, a35, a36, a37, a38, a39, a40, a41, a42, a43, a44, a45)

In [27]:
def  input_tes(x):
    a1, a2, a3, a4, a5, a6, a7, a8, a9 = [x.iloc[250*5*i:256+250*5*i,0:8] for i in range(9)]

    return (a1, a2, a3, a4, a5, a6, a7, a8, a9)

FFT処理

In [28]:
def fft_t01(data, N, fs):
    dt = 1 /fs
    X = np.fft.fft(data)
    freq = np.linspace(0, 1.0/dt, N) # 周波数軸
    amplitudeSpectrum = [np.sqrt(c.real ** 2 + c.imag ** 2) for c in X]
    n = sum(amplitudeSpectrum)
    for i in range(len(amplitudeSpectrum)):
        amplitudeSpectrum[i] = amplitudeSpectrum[i]/n

    amplitudeSpectrum = np.array(amplitudeSpectrum)
    Spectrum =  amplitudeSpectrum[np.logical_and(freq>=3.0, freq<32)]
    am_sum = sum(Spectrum)
    a =  amplitudeSpectrum[np.logical_and(freq>=4, freq<8)]
    a = sum(a)
    p = a / am_sum

    return(p)

def fft_t02(data, N, fs):
    dt = 1 /fs
    X = np.fft.fft(data)
    freq = np.linspace(0, 1.0/dt, N) # 周波数軸
    amplitudeSpectrum = [np.sqrt(c.real ** 2 + c.imag ** 2) for c in X]
    n = sum(amplitudeSpectrum)
    for i in range(len(amplitudeSpectrum)):
        amplitudeSpectrum[i] = amplitudeSpectrum[i]/n

    amplitudeSpectrum = np.array(amplitudeSpectrum)
    Spectrum =  amplitudeSpectrum[np.logical_and(freq>=3.0, freq<32)]
    am_sum = sum(Spectrum)
    a =  amplitudeSpectrum[np.logical_and(freq>=8, freq<13)]
    a = sum(a)
    p = a / am_sum

    return(p)

def fft_t03(data, N, fs):
    dt = 1 /fs
    X = np.fft.fft(data)
    freq = np.linspace(0, 1.0/dt, N) # 周波数軸
    amplitudeSpectrum = [np.sqrt(c.real ** 2 + c.imag ** 2) for c in X]
    n = sum(amplitudeSpectrum)
    for i in range(len(amplitudeSpectrum)):
        amplitudeSpectrum[i] = amplitudeSpectrum[i]/n

    amplitudeSpectrum = np.array(amplitudeSpectrum)
    Spectrum =  amplitudeSpectrum[np.logical_and(freq>=3.0, freq<32)]
    am_sum = sum(Spectrum)
    a =  amplitudeSpectrum[np.logical_and(freq>=13, freq<20)]
    a = sum(a)
    p = a / am_sum

    return(p)

データ分割

In [29]:
def time_separate(data,i):

    data1, data2, data3, data4 = [data[i:i+64] for i in range(0,250,64)]
    data_list = [data1, data2, data3, data4]

    a1 = data1.iloc[:, i]
    a2 = data2.iloc[:, i]
    a3 = data3.iloc[:, i]
    a4 = data4.iloc[:, i]

    x1 = fft_t01(a1,len(a1), 250)
    x2 = fft_t01(a2,len(a2), 250)
    x3 = fft_t01(a3,len(a3), 250)
    x4 = fft_t01(a4,len(a4), 250)

    x5 = fft_t02(a1,len(a1), 250)
    x6 = fft_t02(a2,len(a2), 250)
    x7 = fft_t02(a3,len(a3), 250)
    x8 = fft_t02(a4,len(a4), 250)

    x9 = fft_t03(a1,len(a1), 250)
    x10 = fft_t03(a2,len(a2), 250)
    x11 = fft_t03(a3,len(a3), 250)
    x12 = fft_t03(a4,len(a4), 250)

    return(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12)

訓練データの取得

In [30]:
def get_train(text_data, t, label_list):
    colum_name=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    aso = pd.read_csv(text_data, index_col=0, header=None, names=colum_name)
    aso = aso.ix[6:, :8]
    start = int(t*250)
    aso = aso.ix[start:, :]

    a1, a2, a3, a4, a5, a6, a7, a8, a9,a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31, a32, a33, a34, a35, a36, a37, a38, a39, a40, a41, a42, a43, a44, a45 = input(aso)
    n = [a1, a2, a3, a4, a5, a6, a7, a8, a9,a10, a11, a12, a13, a14, a15, a16, a17, a18, a19, a20, a21, a22, a23, a24, a25, a26, a27, a28, a29, a30, a31, a32, a33, a34, a35, a36, a37, a38, a39, a40, a41, a42, a43, a44, a45]
    features_df = pd.DataFrame()
    features_df['labels'] = label_list
    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,0)
        features_df.loc[i, 'T1_s_1'] = x1
        features_df.loc[i, 'T2_s_1'] = x2
        features_df.loc[i, 'T3_s_1'] = x3
        features_df.loc[i, 'T4_s_1'] = x4
        features_df.loc[i, 'T1_a_1'] = x5
        features_df.loc[i, 'T2_a_1'] = x6
        features_df.loc[i, 'T3_a_1'] = x7
        features_df.loc[i, 'T4_a_1'] = x8
        features_df.loc[i, 'T1_b_1'] = x9
        features_df.loc[i, 'T2_b_1'] = x10
        features_df.loc[i, 'T3_b_1'] = x11
        features_df.loc[i, 'T4_b_1'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,1)
        features_df.loc[i, 'T1_s_2'] = x1
        features_df.loc[i, 'T2_s_2'] = x2
        features_df.loc[i, 'T3_s_2'] = x3
        features_df.loc[i, 'T4_s_2'] = x4
        features_df.loc[i, 'T1_a_2'] = x5
        features_df.loc[i, 'T2_a_2'] = x6
        features_df.loc[i, 'T3_a_2'] = x7
        features_df.loc[i, 'T4_a_2'] = x8
        features_df.loc[i, 'T1_b_2'] = x9
        features_df.loc[i, 'T2_b_2'] = x10
        features_df.loc[i, 'T3_b_2'] = x11
        features_df.loc[i, 'T4_b_2'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,2)
        features_df.loc[i, 'T1_s_3'] = x1
        features_df.loc[i, 'T2_s_3'] = x2
        features_df.loc[i, 'T3_s_3'] = x3
        features_df.loc[i, 'T4_s_3'] = x4
        features_df.loc[i, 'T1_a_3'] = x5
        features_df.loc[i, 'T2_a_3'] = x6
        features_df.loc[i, 'T3_a_3'] = x7
        features_df.loc[i, 'T4_a_3'] = x8
        features_df.loc[i, 'T1_b_3'] = x9
        features_df.loc[i, 'T2_b_3'] = x10
        features_df.loc[i, 'T3_b_3'] = x11
        features_df.loc[i, 'T4_b_3'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,3)
        features_df.loc[i, 'T1_s_4'] = x1
        features_df.loc[i, 'T2_s_4'] = x2
        features_df.loc[i, 'T3_s_4'] = x3
        features_df.loc[i, 'T4_s_4'] = x4
        features_df.loc[i, 'T1_a_4'] = x5
        features_df.loc[i, 'T2_a_4'] = x6
        features_df.loc[i, 'T3_a_4'] = x7
        features_df.loc[i, 'T4_a_4'] = x8
        features_df.loc[i, 'T1_b_4'] = x9
        features_df.loc[i, 'T2_b_4'] = x10
        features_df.loc[i, 'T3_b_4'] = x11
        features_df.loc[i, 'T4_b_4'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,4)
        features_df.loc[i, 'T1_s_5'] = x1
        features_df.loc[i, 'T2_s_5'] = x2
        features_df.loc[i, 'T3_s_5'] = x3
        features_df.loc[i, 'T4_s_5'] = x4
        features_df.loc[i, 'T1_a_5'] = x5
        features_df.loc[i, 'T2_a_5'] = x6
        features_df.loc[i, 'T3_a_5'] = x7
        features_df.loc[i, 'T4_a_5'] = x8
        features_df.loc[i, 'T1_b_5'] = x9
        features_df.loc[i, 'T2_b_5'] = x10
        features_df.loc[i, 'T3_b_5'] = x11
        features_df.loc[i, 'T4_b_5'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,5)
        features_df.loc[i, 'T1_s_6'] = x1
        features_df.loc[i, 'T2_s_6'] = x2
        features_df.loc[i, 'T3_s_6'] = x3
        features_df.loc[i, 'T4_s_6'] = x4
        features_df.loc[i, 'T1_a_6'] = x5
        features_df.loc[i, 'T2_a_6'] = x6
        features_df.loc[i, 'T3_a_6'] = x7
        features_df.loc[i, 'T4_a_6'] = x8
        features_df.loc[i, 'T1_b_6'] = x9
        features_df.loc[i, 'T2_b_6'] = x10
        features_df.loc[i, 'T3_b_6'] = x11
        features_df.loc[i, 'T4_b_6'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,6)
        features_df.loc[i, 'T1_s_7'] = x1
        features_df.loc[i, 'T2_s_7'] = x2
        features_df.loc[i, 'T3_s_7'] = x3
        features_df.loc[i, 'T4_s_7'] = x4
        features_df.loc[i, 'T1_a_7'] = x5
        features_df.loc[i, 'T2_a_7'] = x6
        features_df.loc[i, 'T3_a_7'] = x7
        features_df.loc[i, 'T4_a_7'] = x8
        features_df.loc[i, 'T1_b_7'] = x9
        features_df.loc[i, 'T2_b_7'] = x10
        features_df.loc[i, 'T3_b_7'] = x11
        features_df.loc[i, 'T4_b_7'] = x12

    for i, j in zip(range(len(n)), n):
        x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = time_separate(j,7)
        features_df.loc[i, 'T1_s_8'] = x1
        features_df.loc[i, 'T2_s_8'] = x2
        features_df.loc[i, 'T3_s_8'] = x3
        features_df.loc[i, 'T4_s_8'] = x4
        features_df.loc[i, 'T1_a_8'] = x5
        features_df.loc[i, 'T2_a_8'] = x6
        features_df.loc[i, 'T3_a_8'] = x7
        features_df.loc[i, 'T4_a_8'] = x8
        features_df.loc[i, 'T1_b_8'] = x9
        features_df.loc[i, 'T2_b_8'] = x10
        features_df.loc[i, 'T3_b_8'] = x11
        features_df.loc[i, 'T4_b_8'] = x12


    #X_train = features_df.ix[:, 1:]
    #y_train = features_df.ix[:, 0]

    return(features_df)

教師データの入力

In [31]:
list01 = [1,2,0,1,1,0,2,1,2,1,2,0,2,1,0,1,2,0,1,2,1,0,1,0,2,1,0,1,2,1,0,0,1,2,1,0,2,1,0,2,1,2,0,2,1]
list02 = [1,2,0,0,2,1,2,1,2,1,0,0,2,1,2,1,2,0,2,1,0,2,1,0,1,2,0,1,0,1,2,1,0,1,0,1,2,1,2,0,0,2,0,1,0]
list03 = [1,0,2,1,0,2,2,1,2,0,1,2,0,1,2,1,0,1,0,2,1,0,2,2,1,1,0,0,1,0,2,2,0,1,2,1,2,0,1,2,0,1,2,0,1]

mainの処理

In [32]:
feature_df01 = get_train("OpenBCI-RAW-ita01.txt", 6.67, list01)
feature_df02 = get_train("OpenBCI-RAW-ita02.txt", 7.14, list02)
feature_df03 = get_train("OpenBCI-RAW-ita03.txt", 7.52, list03)

In [36]:
train_feature = pd.concat([feature_df01, feature_df02, feature_df03])
train_feature = train_feature.sample(frac=1, random_state=0).reset_index(drop=True)

X_train = train_feature.ix[:119, 1:]
y_train = train_feature.ix[:119, 0]

X_test = train_feature.ix[120:134, 1:]
y_test = train_feature.ix[120:134, 0]

pca = PCA(n_components=30)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

#print('累積寄与率: {0}'.format(sum(pca.explained_variance_ratio_)))


clf=svm.SVC(class_weight='balanced', random_state=0)
param_range=[0.01, 0.1, 1.0] #変化させるパラメータに設定する値たち
param_grid=[{'C':param_range,'kernel':['rbf', 'linear'], 'gamma':param_range}] #Cとカーネルとgammaを変化させて最適化させる

# 解説6：グリッドサーチにより最適値を求める
gs=grid_search.GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=-1)
gs=gs.fit(X_train,y_train)
print(gs.best_score_)
print(gs.best_params_)


clf_result=svm.SVC(kernel='rbf', gamma=1.0 , C=1.0,class_weight='balanced', random_state=0) #loss='squared_hinge' #loss="hinge", loss="log"
clf_result.fit(X_train, y_train)

pre=clf_result.predict(X_test)
ac_score=metrics.accuracy_score(y_test,pre)
print("正答率 = ",ac_score)

0.375
{'gamma': 1.0, 'C': 1.0, 'kernel': 'rbf'}
正答率 =  0.4


In [44]:
train_feature = train_feature.sample(frac=1, random_state=0).reset_index(drop=True)
train_feature

,labels,T1_s_1,T2_s_1,T3_s_1,T4_s_1,T1_a_1,T2_a_1,T3_a_1,T4_a_1,T1_b_1,...,T3_s_8,T4_s_8,T1_a_8,T2_a_8,T3_a_8,T4_a_8,T1_b_8,T2_b_8,T3_b_8,T4_b_8
0,1,0.150883,0.160975,0.070137,0.207446,0.125502,0.120502,0.100576,0.112767,0.172402,...,0.120693,0.103332,0.114080,0.115846,0.110131,0.125097,0.212665,0.219895,0.238019,0.255576
1,2,0.247313,0.240297,0.109061,0.239418,0.108072,0.192637,0.160564,0.075454,0.161685,...,0.101071,0.121867,0.114654,0.040753,0.131264,0.115651,0.227926,0.157093,0.246202,0.251494
2,0,0.060918,0.059923,0.119837,0.082232,0.086927,0.104724,0.092398,0.120307,0.180305,...,0.103802,0.116288,0.111535,0.111656,0.091253,0.115286,0.237426,0.247621,0.230776,0.240946
3,2,0.149084,0.175999,0.169635,0.165851,0.105577,0.112401,0.117353,0.139432,0.144251,...,0.127696,0.095287,0.029165,0.117457,0.099557,0.118038,0.069836,0.227942,0.235533,0.245509
4,1,0.135234,0.129597,0.106547,0.108883,0.097900,0.140161,0.106575,0.109878,0.216388,...,0.105118,0.057304,0.091385,0.113074,0.120231,0.060465,0.212977,0.239509,0.245914,0.219098
5,1,0.103977,0.128525,0.049900,0.098281,0.117688,0.117401,0.079507,0.121241,0.243218,...,0.106074,0.109775,0.096445,0.097492,0.121194,0.113884,0.230388,0.240955,0.246914,0.238036
6,1,0.119217,0.108083,0.108027,0.100813,0.102628,0.126070,0.085679,0.129324,0.253157,...,0.111572,0.074655,0.109265,0.110139,0.111794,0.089364,0.177528,0.225875,0.250331,0.204558
7,1,0.214010,0.115721,0.105820,0.183026,0.075651,0.088638,0.150562,0.102804,0.073032,...,0.106531,0.127373,0.112259,0.095262,0.117390,0.093573,0.242172,0.234215,0.237039,0.246878
8,0,0.194644,0.181039,0.065976,0.186931,0.131930,0.079414,0.258118,0.059629,0.104484,...,0.105779,0.119019,0.121012,0.109403,0.099739,0.120546,0.243005,0.234321,0.225279,0.235550
9,2,0.127264,0.137057,0.127741,0.067155,0.116266,0.101100,0.075846,0.099369,0.226326,...,0.120523,0.099351,0.090086,0.112740,0.107371,0.099612,0.232583,0.247419,0.234333,0.228435


In [40]:
y_test

120    0
121    0
122    1
123    2
124    1
125    1
126    2
127    0
128    2
129    0
130    1
131    1
132    1
133    0
134    0
Name: labels, dtype: int64